---
title: "Illustrating Model Properties"
---


# Diagnose Time Prior and Evolution of Midline Extension

First, we illustrate how the prior distribution over diagnose times and the conditional distribution of having developed a tumor that crosses the mid-sagittal line *given* a diagnosis time evolve together.


In [ ]:
#| label: fig-model-midext-evo
#| fig-cap: 'The top panel shows the prior probability to get diagnosed at time-step $t$ for early and late T-category tumors as bars. Also in the top panel, we plot the conditional probability of the tumor''s midline extension ($\epsilon=\texttt{True}$), given the time-step $t$ as a line plot. In the bottom panel, we show the joint probability of getting diagnosed in time-step $t$ *and* having a tumor that crosses the midline.'

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from lyscripts.plot.utils import COLORS
from scripts import shared

midline_model = shared.get_model("midline", load_samples=True)

t = np.linspace(0, 10, 11)
p_midline = {
  "lateralized": (1 - midline_model.midext_prob)**t,
  "extension": 1 - (1 - midline_model.midext_prob)**t,
}
dist = {
  "early": midline_model.get_distribution("early").pmf,
  "late": midline_model.get_distribution("late").pmf,
}
p_colors = {"lateralized": COLORS["green"], "extension": COLORS["red"]}
t_colors = {"early": COLORS["blue"], "late": COLORS["orange"]}
markers = {}

nrows, ncols = 2, 1
plt.rcParams.update(shared.get_fontsizes())
plt.rcParams.update(shared.get_figsizes(
  nrows=nrows,
  ncols=ncols,
  width=17/2,
  aspect_ratio=2.5,
))

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True)
w = 0.3

for label, p in p_midline.items():
  axes[0].plot(
    t, p_midline[label], "o-",
    label=f"cond. prob. $P(\epsilon={label=='extension'} \mid t)$",
    color=p_colors[label],
  )
  for i, t_stage in enumerate(["early", "late"]):
    if label == "lateralized":
      axes[0].bar(
        t + i*w - w/2, dist[t_stage],
        color=t_colors[t_stage],
        width=w,
        label=f"{t_stage} T-cat. prior $P(t)$",
      )
    axes[1].plot(
      t, p * dist[t_stage], "o-",
      c=p_colors[label],
      mfc=t_colors[t_stage],
      mec=t_colors[t_stage],
      label=f"{label} ($\epsilon={label=='extension'}$) for {t_stage} T-cat.",
    )

axes[0].set_ylim(0., 1.)
axes[0].yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.0%}"))
axes[0].set_ylabel("probability")
axes[0].grid(axis="y", color=COLORS["gray"], alpha=0.5)

axes[0].legend()

axes[1].set_xlim(min(t), max(t))
axes[1].set_xticks(t)
axes[1].set_xlabel("time-step $t$")

axes[1].set_ylim(0., 0.25)
axes[1].yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.0%}"))
axes[1].set_ylabel("joint probability $P(\epsilon, t)$")
axes[1].grid(axis="y", color=COLORS["gray"], alpha=0.5)

axes[1].legend()

plt.show()

# Sampling History

For bookeeping, we also show how different quantities evolve during the burnin of the sampling. For example, the ratio of proposed samples that were accepted or the autocorrelation time.


In [ ]:
#| echo: false
#| label: fig-model-burnin-history
#| fig-cap: Monitoring quantities during the burn-in phase of the parameter sampling.
from collections import namedtuple
import pandas as pd

def custom(x, pos):
  return "{:.1f}".format(x/1000)

nrows, ncols = 1, 3
label_map = {
  "acor_times": "autocorrelation [steps]",
  "accept_fracs": "acceptance fraction [%]",
  "max_log_probs": r"max. log-likelihood [$\times 10^3$]",
}

plt.rcParams.update(shared.get_fontsizes())
plt.rcParams.update(shared.get_figsizes(
  nrows=nrows,
  ncols=ncols,
  aspect_ratio=1.,
  width=17,
))

fig, axes = plt.subplots(
  nrows=nrows,
  ncols=ncols,
  sharex=True,
)

for model in ["contra", "bilateral", "midline"]:
  history = pd.read_csv(f"models/{model}/history.csv").set_index("steps")
  for i, column in enumerate(history.columns):
    history.plot(y=column, ax=axes[i], label=model)
    axes[i].autoscale(enable=True, tight=True, axis="x")
    axes[i].set_ylabel(label_map[column])
    axes[i].xaxis.set_major_formatter(ticker.FuncFormatter(custom))
    axes[i].set_xlabel(r"steps [$\times 10^3$]")

axes[1].yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.0%}"))
axes[2].yaxis.set_major_formatter(ticker.FuncFormatter(custom))
plt.show()